# Built-in Runnables in LCEL

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [1]:
!pip install -U langchain-community langchain langchain-huggingface huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN= userdata.get('HUGGING_FACE_API_KEY')

from huggingface_hub import login
login(token = HUGGINGFACEHUB_API_TOKEN)

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
import transformers, torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token  # Set pad_token to eos_token

model_kwargs = {'temperature':0.3}
model = HuggingFacePipeline(pipeline = pipe, model_kwargs = model_kwargs)

chatModel= ChatHuggingFace(llm=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


## RunnablePassthrough
* It does not do anything to the input data.
* Let's see it in a very simple example: a chain with just **`RunnablePassthrough()`** will output the original input without any modification.

In [3]:
from langchain_core.runnables import RunnablePassthrough
chain= RunnablePassthrough()

In [5]:
chain.invoke("Mohammad Wasiq")

'Mohammad Wasiq'

## RunnableLambda
* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
* Let's define a very simple function to create Russian lastnames:

In [7]:
def lastname(name: str) -> str:
    return f"{name} QCS"

In [8]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(lastname)

In [9]:
chain.invoke("Mohammad Wasiq")

'Mohammad Wasiq QCS'

## RunnableParallel
* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
    * operation_a will use RunnablePassthrough.
    * operation_b will use RunnableLambda with the russian_lastname function.

In [10]:
from langchain_core.runnables import RunnableParallel

In [11]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(lastname)
    }
)

In [12]:
chain.invoke("Mohammad Wasiq")

{'operation_a': 'Mohammad Wasiq', 'operation_b': 'Mohammad Wasiq QCS'}

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [14]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [17]:
def lastname_from_dictionary(person):
    return person["name"] + "Pasha Turk"

In [18]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }


) | prompt | model | output_parser

In [19]:
chain.invoke({
    "name1": "Zainul Abedeen",
     "name": "Altamash"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Human: tell me a curious fact about AltamashPasha Turkmenistan\nAltamashPasha is a small town in the Turkmenistan region of Central Asia. Here's a curious fact:\nAltamashPasha is home to the ancient city of Derbent, which was once an important center of trade and culture in the region. The city is situated on the banks of the Amu Darya River and is known for its well-preserved ancient architecture, including the remains of the Derbent Fortress, which dates back to the 6th century BC. The city has been an important cultural and economic hub for centuries, and its strategic location made it a key stopover for merchants and travelers on the Silk Road. Today, AltamashPasha is a popular tourist destination, attracting visitors from around the world who come to explore its rich history and cultural heritage."

In [21]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings


In [24]:
%pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00


In [25]:
vectorstore = FAISS.from_texts(
    ["learning logic  focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=HuggingFaceEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# model = ChatOpenAI(model="gpt-3.5-turbo")

chatModel= ChatHuggingFace(llm=model)

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | chatModel
    | StrOutputParser()
)
retrieval_chain.invoke("What is is learning logic ")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nAnswer the question based only on the following context:\n[Document(id=\'8e1f6db6-29e9-4233-996b-a2b88faedc81\', metadata={}, page_content=\'learning logic  focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.\')]\n\nQuestion: What is is learning logic<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nBased on the provided context, it appears that "learning logic" is likely referring to a type of algorithm or approach used in machine learning and artificial intelligence (AI).'